### Develop a classification model using YOLO object detection algorithm using OpenCV. 

In [40]:
import cv2
import numpy as np

net=cv2.dnn.readNet("yolov3.weights","yolov3.cfg")
layer_names=net.getLayerNames()
output_layers=[layer_names[i-1] for i in net.getUnconnectedOutLayers()]

In [41]:
with open("coco.names","r") as f:
    classes=[line.strip() for line in f.readlines()]

In [42]:
image=cv2.imread('../images/img.png')
height,width=image.shape[:2]

In [43]:
blob=cv2.dnn.blobFromImage(image,1/255.0,(416,416),(0,0,0),True,crop=False)
net.setInput(blob)
outs=net.forward(output_layers)

In [47]:
detected_boxes=[]
detected_classes=[]
detected_confidences=[]

In [48]:
for out in outs:
    for detection in out:
        scores=detection[5:]
        class_id=np.argmax(scores)
        confidence=scores[class_id]
        if confidence>0.2:
            center_x,center_y,width,height=(detection[0:4]*np.array([width,height,width,height])).astype("int")
            x=int(center_x-width/2)
            y=int(center_y-height/2)
            detected_boxes.append([x,y,int(width),int(height)])
            detected_classes.append(class_id)
            detected_confidences.append(float(confidence))

In [49]:
indices=cv2.dnn.NMSBoxes(detected_boxes,detected_confidences,0.5,0.3)
print(indices)
for i in indices.flatten():
    x,y,w,h=detected_boxes[i]
    label=f"{classes[detected_classes[i]]}:{detected_confidences[i]:.2f}"
    cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0,2))
    cv2.putText(image,classes[detected_classes[i]],(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2)

[0]


In [50]:
cv2.imshow("YOLO object detection",image)
cv2.waitKey(0)
cv2.destroyAllWindows()